# A/B Test Campaign Funnel Analysis
## 📊 Statistical Analysis of Control vs Test Campaign Performance

### Analysis Objectives:
- Compare campaign performance between Control and Test groups
- Statistical hypothesis testing for conversion rates
- Funnel analysis to identify optimization opportunities
- Data-driven recommendations for campaign strategy

## 1. Data Preparation & Quality Assessment

In [1]:
# Essential libraries for comprehensive data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Statistical testing libraries
from scipy import stats
from scipy.stats import ttest_ind, mannwhitneyu, chi2_contingency
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Set style for better visualization
plt.style.use('seaborn')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

In [2]:
def load_and_preprocess_data():
    """
    Load and preprocess campaign data with comprehensive data quality checks
    """
    # Column mapping for consistent naming
    column_names = ['campaign_name', 'date', 'spend', 'impression', 'reach', 
                   'click', 'search', 'view', 'cart', 'purchase']
    
    # Load Control Group
    control_df = pd.read_csv("control_group.csv", sep=";")
    control_df['Date'] = pd.to_datetime(control_df['Date'], format='%d.%m.%Y')
    control_df.columns = column_names
    control_df['campaign_name'] = 'control'
    
    # Load Test Group  
    test_df = pd.read_csv("test_group.csv", sep=";")
    test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d.%m.%Y')
    test_df.columns = column_names
    test_df['campaign_name'] = 'test'
    
    return control_df, test_df

def data_quality_assessment(df, campaign_name):
    """
    Comprehensive data quality assessment
    """
    print(f"\n=== {campaign_name.upper()} CAMPAIGN DATA QUALITY ===\n")
    
    # Basic info
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"📅 Date Range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"⏱️  Duration: {(df['date'].max() - df['date'].min()).days + 1} days")
    
    # Missing data analysis
    missing_data = df.isnull().sum()
    missing_percent = 100 * missing_data / len(df)
    
    if missing_data.sum() > 0:
        print("\n🚨 Missing Data Detected:")
        for col in missing_data[missing_data > 0].index:
            print(f"   {col}: {missing_data[col]} rows ({missing_percent[col]:.1f}%)")
    else:
        print("\n✅ No missing data detected")
    
    # Statistical summary for key metrics
    key_metrics = ['spend', 'impression', 'click', 'purchase']
    print("\n📈 Key Metrics Summary:")
    summary_stats = df[key_metrics].describe()
    print(summary_stats.round(2))
    
    return {
        'missing_data': missing_data,
        'summary_stats': summary_stats,
        'date_range': (df['date'].min(), df['date'].max())
    }

# Load data
control_df, test_df = load_and_preprocess_data()

# Data quality assessment
control_quality = data_quality_assessment(control_df, 'Control')
test_quality = data_quality_assessment(test_df, 'Test')


=== CONTROL CAMPAIGN DATA QUALITY ===

📊 Dataset Shape: (30, 10)
📅 Date Range: 2019-08-01 to 2019-08-30
⏱️  Duration: 30 days

🚨 Missing Data Detected:
   impression: 1 rows (3.3%)
   reach: 1 rows (3.3%)
   click: 1 rows (3.3%)
   search: 1 rows (3.3%)
   view: 1 rows (3.3%)
   cart: 1 rows (3.3%)
   purchase: 1 rows (3.3%)

📈 Key Metrics Summary:
         spend  impression    click  purchase
count    30.00       29.00    29.00     29.00
mean   2288.43   109559.76  5320.79    522.79
std     367.33    21688.92  1757.37    185.03
min    1757.00    71274.00  2277.00    222.00
25%    1945.50    92029.00  4085.00    372.00
50%    2299.50   113430.00  5224.00    501.00
75%    2532.00   121332.00  6628.00    670.00
max    3083.00   145248.00  8137.00    800.00

=== TEST CAMPAIGN DATA QUALITY ===

📊 Dataset Shape: (30, 10)
📅 Date Range: 2019-08-01 to 2019-08-30
⏱️  Duration: 30 days

✅ No missing data detected

📈 Key Metrics Summary:
         spend  impression    click  purchase
count    30.

## 2. Missing Data Treatment & Sensitivity Analysis
### Multiple scenarios to test robustness of conclusions

In [3]:
def create_analysis_scenarios(control_df, test_df):
    """
    Create multiple scenarios for sensitivity analysis
    """
    scenarios = {}
    
    # Identify numeric columns
    numeric_cols = control_df.select_dtypes(include=[np.number]).columns
    
    # Scenario 1: Remove rows with missing data
    control_clean = control_df.dropna()
    test_clean = test_df.dropna()
    
    # Only include dates present in both datasets
    common_dates = set(control_clean['date']) & set(test_clean['date'])
    control_s1 = control_clean[control_clean['date'].isin(common_dates)]
    test_s1 = test_clean[test_clean['date'].isin(common_dates)]
    
    scenarios['complete_cases'] = pd.concat([control_s1, test_s1], ignore_index=True)
    
    # Scenario 2: Fill with median (more robust to outliers)
    control_median = control_df.copy()
    control_median[numeric_cols] = control_median[numeric_cols].fillna(control_median[numeric_cols].median())
    scenarios['median_fill'] = pd.concat([control_median, test_df], ignore_index=True)
    
    # Scenario 3: Fill with mean
    control_mean = control_df.copy()
    control_mean[numeric_cols] = control_mean[numeric_cols].fillna(control_mean[numeric_cols].mean())
    scenarios['mean_fill'] = pd.concat([control_mean, test_df], ignore_index=True)
    
    return scenarios

# Create analysis scenarios
analysis_scenarios = create_analysis_scenarios(control_df, test_df)

print("📋 Analysis Scenarios Created:")
for name, df in analysis_scenarios.items():
    control_rows = len(df[df['campaign_name'] == 'control'])
    test_rows = len(df[df['campaign_name'] == 'test'])
    print(f"   {name}: Control={control_rows}, Test={test_rows}, Total={len(df)}")

# Use the most conservative scenario (complete cases) for primary analysis
primary_df = analysis_scenarios['complete_cases'].copy()
print(f"\n✅ Primary analysis will use: complete_cases scenario")
print(f"📊 Final dataset shape: {primary_df.shape}")

📋 Analysis Scenarios Created:
   complete_cases: Control=29, Test=29, Total=58
   median_fill: Control=30, Test=30, Total=60
   mean_fill: Control=30, Test=30, Total=60

✅ Primary analysis will use: complete_cases scenario
📊 Final dataset shape: (58, 10)


## 3. Exploratory Data Analysis & Campaign Overview

In [4]:
def comprehensive_eda(df):
    """
    Comprehensive exploratory data analysis for campaign performance
    """
    print("\n" + "="*60)
    print("           📊 CAMPAIGN PERFORMANCE OVERVIEW")
    print("="*60)
    
    # Campaign-level aggregated metrics
    campaign_summary = df.groupby('campaign_name').agg({
        'spend': ['sum', 'mean', 'std'],
        'impression': ['sum', 'mean'],
        'click': ['sum', 'mean'],
        'search': ['sum', 'mean'],
        'view': ['sum', 'mean'],
        'cart': ['sum', 'mean'],
        'purchase': ['sum', 'mean']
    }).round(2)
    
    print("\n🎯 Campaign Summary Statistics:")
    print(campaign_summary)
    
    # Calculate key performance indicators
    kpi_summary = df.groupby('campaign_name').apply(calculate_kpis).round(4)
    print("\n📈 Key Performance Indicators:")
    print(kpi_summary)
    
    return campaign_summary, kpi_summary

def calculate_kpis(group_df):
    """
    Calculate comprehensive KPIs for campaign analysis
    """
    total_spend = group_df['spend'].sum()
    total_impressions = group_df['impression'].sum()
    total_clicks = group_df['click'].sum()
    total_purchases = group_df['purchase'].sum()
    total_views = group_df['view'].sum()
    total_carts = group_df['cart'].sum()
    
    return pd.Series({
        'CTR': (total_clicks / total_impressions) * 100,  # Click-through rate (%)
        'CPC': total_spend / total_clicks,  # Cost per click
        'CPM': (total_spend / total_impressions) * 1000,  # Cost per mille
        'CPA': total_spend / total_purchases,  # Cost per acquisition
        'Conversion_Rate': (total_purchases / total_clicks) * 100,  # Purchase conversion rate (%)
        'View_to_Cart': (total_carts / total_views) * 100,  # View to cart rate (%)
        'Cart_to_Purchase': (total_purchases / total_carts) * 100,  # Cart to purchase rate (%)
        'ROAS_Proxy': total_purchases / total_spend,  # Purchases per dollar spent
    })

# Perform comprehensive EDA
campaign_summary, kpi_summary = comprehensive_eda(primary_df)


           📊 CAMPAIGN PERFORMANCE OVERVIEW

🎯 Campaign Summary Statistics:
               spend                  impression                click  \
                 sum     mean     std        sum       mean       sum   
campaign_name                                                           
control        66818  2304.07  363.53  3177233.0  109559.76  154303.0   
test           74595  2572.24  351.16  2123249.0   73215.48  175107.0   

                         search              view              cart           \
                  mean      sum     mean      sum     mean      sum     mean   
campaign_name                                                                  
control        5320.79  64418.0  2221.31  56370.0  1943.79  37700.0  1300.00   
test           6038.17  70463.0  2429.76  54882.0  1892.48  25490.0   878.97   

              purchase          
                   sum    mean  
campaign_name                   
control        15161.0  522.79  
test           14869.0  5

## 4. Statistical Hypothesis Testing Framework

In [5]:
def statistical_testing_suite(df, alpha=0.05):
    """
    Comprehensive statistical testing framework for A/B testing
    """
    print("\n" + "="*60)
    print("           🔬 STATISTICAL HYPOTHESIS TESTING")
    print("="*60)
    print(f"Significance Level (α): {alpha}")
    
    control_data = df[df['campaign_name'] == 'control']
    test_data = df[df['campaign_name'] == 'test']
    
    results = {}
    
    # 1. Purchase Conversion Rate Test (Primary Metric)
    print("\n🎯 PRIMARY METRIC: Purchase Conversion Rate")
    control_conversions = control_data['purchase'].sum()
    control_clicks = control_data['click'].sum()
    test_conversions = test_data['purchase'].sum()
    test_clicks = test_data['click'].sum()
    
    # Two-proportion z-test for conversion rates
    count = np.array([control_conversions, test_conversions])
    nobs = np.array([control_clicks, test_clicks])
    
    z_stat, p_value = proportions_ztest(count, nobs)
    
    # Confidence intervals
    control_ci = proportion_confint(control_conversions, control_clicks, alpha=alpha)
    test_ci = proportion_confint(test_conversions, test_clicks, alpha=alpha)
    
    control_rate = control_conversions / control_clicks
    test_rate = test_conversions / test_clicks
    lift = (test_rate - control_rate) / control_rate * 100
    
    results['conversion_rate'] = {
        'control_rate': control_rate,
        'test_rate': test_rate,
        'lift_percent': lift,
        'z_statistic': z_stat,
        'p_value': p_value,
        'significant': p_value < alpha,
        'control_ci': control_ci,
        'test_ci': test_ci
    }
    
    print(f"   Control Rate: {control_rate:.4f} ({control_rate*100:.2f}%)")
    print(f"   Test Rate: {test_rate:.4f} ({test_rate*100:.2f}%)")
    print(f"   Lift: {lift:+.2f}%")
    print(f"   Z-statistic: {z_stat:.4f}")
    print(f"   P-value: {p_value:.6f}")
    print(f"   Significant: {'✅ YES' if p_value < alpha else '❌ NO'} (α = {alpha})")
    
    # 2. Cost Per Acquisition Test
    print("\n💰 SECONDARY METRIC: Cost Per Acquisition")
    control_cpa = control_data['spend'].sum() / control_conversions
    test_cpa = test_data['spend'].sum() / test_conversions
    
    # Daily CPA for statistical testing
    control_daily_cpa = (control_data['spend'] / control_data['purchase']).replace([np.inf, -np.inf], np.nan).dropna()
    test_daily_cpa = (test_data['spend'] / test_data['purchase']).replace([np.inf, -np.inf], np.nan).dropna()
    
    if len(control_daily_cpa) > 1 and len(test_daily_cpa) > 1:
        # Check normality
        control_normal = stats.shapiro(control_daily_cpa)[1] > 0.05
        test_normal = stats.shapiro(test_daily_cpa)[1] > 0.05
        
        if control_normal and test_normal:
            stat, p_val = stats.ttest_ind(control_daily_cpa, test_daily_cpa)
            test_type = "Independent t-test"
        else:
            stat, p_val = stats.mannwhitneyu(control_daily_cpa, test_daily_cpa)
            test_type = "Mann-Whitney U test"
        
        cpa_lift = (test_cpa - control_cpa) / control_cpa * 100
        
        results['cpa'] = {
            'control_cpa': control_cpa,
            'test_cpa': test_cpa,
            'lift_percent': cpa_lift,
            'test_type': test_type,
            'statistic': stat,
            'p_value': p_val,
            'significant': p_val < alpha
        }
        
        print(f"   Control CPA: ${control_cpa:.2f}")
        print(f"   Test CPA: ${test_cpa:.2f}")
        print(f"   Change: {cpa_lift:+.2f}%")
        print(f"   Test Used: {test_type}")
        print(f"   P-value: {p_val:.6f}")
        print(f"   Significant: {'✅ YES' if p_val < alpha else '❌ NO'}")
    
    # 3. Click-Through Rate Test
    print("\n👆 ENGAGEMENT METRIC: Click-Through Rate")
    control_impressions = control_data['impression'].sum()
    test_impressions = test_data['impression'].sum()
    
    ctr_count = np.array([control_clicks, test_clicks])
    ctr_nobs = np.array([control_impressions, test_impressions])
    
    ctr_z_stat, ctr_p_value = proportions_ztest(ctr_count, ctr_nobs)
    
    control_ctr = control_clicks / control_impressions
    test_ctr = test_clicks / test_impressions
    ctr_lift = (test_ctr - control_ctr) / control_ctr * 100
    
    results['ctr'] = {
        'control_ctr': control_ctr,
        'test_ctr': test_ctr,
        'lift_percent': ctr_lift,
        'z_statistic': ctr_z_stat,
        'p_value': ctr_p_value,
        'significant': ctr_p_value < alpha
    }
    
    print(f"   Control CTR: {control_ctr:.4f} ({control_ctr*100:.2f}%)")
    print(f"   Test CTR: {test_ctr:.4f} ({test_ctr*100:.2f}%)")
    print(f"   Lift: {ctr_lift:+.2f}%")
    print(f"   P-value: {ctr_p_value:.6f}")
    print(f"   Significant: {'✅ YES' if ctr_p_value < alpha else '❌ NO'}")
    
    return results

# Perform statistical testing
test_results = statistical_testing_suite(primary_df)


           🔬 STATISTICAL HYPOTHESIS TESTING
Significance Level (α): 0.05

🎯 PRIMARY METRIC: Purchase Conversion Rate
   Control Rate: 0.0983 (9.83%)
   Test Rate: 0.0849 (8.49%)
   Lift: -13.58%
   Z-statistic: 13.2741
   P-value: 0.000000
   Significant: ✅ YES (α = 0.05)

💰 SECONDARY METRIC: Cost Per Acquisition
   Control CPA: $4.41
   Test CPA: $5.02
   Change: +13.83%
   Test Used: Mann-Whitney U test
   P-value: 0.196788
   Significant: ❌ NO

👆 ENGAGEMENT METRIC: Click-Through Rate
   Control CTR: 0.0486 (4.86%)
   Test CTR: 0.0825 (8.25%)
   Lift: +69.82%
   P-value: 0.000000
   Significant: ✅ YES


## 5. Funnel Analysis & Conversion Path Optimization

In [6]:
def funnel_analysis(df):
    """
    Comprehensive funnel analysis with step-by-step conversion rates
    """
    print("\n" + "="*60)
    print("           🔄 CAMPAIGN FUNNEL ANALYSIS")
    print("="*60)
    
    # Aggregate funnel metrics by campaign
    funnel_data = df.groupby('campaign_name').agg({
        'impression': 'sum',
        'click': 'sum', 
        'search': 'sum',
        'view': 'sum',
        'cart': 'sum',
        'purchase': 'sum'
    }).reset_index()
    
    # Calculate conversion rates for each step
    def calculate_funnel_rates(row):
        rates = {
            'campaign': row['campaign_name'],
            'impressions': int(row['impression']),
            'clicks': int(row['click']),
            'searches': int(row['search']),
            'views': int(row['view']),
            'carts': int(row['cart']),
            'purchases': int(row['purchase']),
            
            # Conversion rates
            'impression_to_click': row['click'] / row['impression'],
            'click_to_search': row['search'] / row['click'],
            'search_to_view': row['view'] / row['search'],
            'view_to_cart': row['cart'] / row['view'],
            'cart_to_purchase': row['purchase'] / row['cart'],
            
            # Overall conversion rate
            'overall_conversion': row['purchase'] / row['impression']
        }
        return pd.Series(rates)
    
    funnel_rates = funnel_data.apply(calculate_funnel_rates, axis=1)
    
    print("\n📊 Funnel Conversion Rates:")
    print("\n" + "-"*80)
    
    for _, row in funnel_rates.iterrows():
        campaign = row['campaign'].upper()
        print(f"\n🎯 {campaign} CAMPAIGN:")
        print(f"   Impressions → Clicks:    {row['impression_to_click']*100:6.2f}%  ({row['impressions']:,} → {row['clicks']:,})")
        print(f"   Clicks → Searches:       {row['click_to_search']*100:6.2f}%  ({row['clicks']:,} → {row['searches']:,})")
        print(f"   Searches → Views:        {row['search_to_view']*100:6.2f}%  ({row['searches']:,} → {row['views']:,})")
        print(f"   Views → Carts:           {row['view_to_cart']*100:6.2f}%  ({row['views']:,} → {row['carts']:,})")
        print(f"   Carts → Purchases:       {row['cart_to_purchase']*100:6.2f}%  ({row['carts']:,} → {row['purchases']:,})")
        print(f"   " + "-"*50)
        print(f"   Overall Conversion:      {row['overall_conversion']*100:6.4f}%  ({row['impressions']:,} → {row['purchases']:,})")
    
    # Calculate lift for each funnel step
    control_row = funnel_rates[funnel_rates['campaign'] == 'control'].iloc[0]
    test_row = funnel_rates[funnel_rates['campaign'] == 'test'].iloc[0]
    
    print("\n🚀 FUNNEL STEP PERFORMANCE COMPARISON:")
    print("\n" + "-"*80)
    
    funnel_steps = [
        ('Impression → Click', 'impression_to_click'),
        ('Click → Search', 'click_to_search'),
        ('Search → View', 'search_to_view'),
        ('View → Cart', 'view_to_cart'),
        ('Cart → Purchase', 'cart_to_purchase'),
        ('Overall Conversion', 'overall_conversion')
    ]
    
    funnel_comparison = []
    
    for step_name, metric in funnel_steps:
        control_rate = control_row[metric]
        test_rate = test_row[metric]
        lift = (test_rate - control_rate) / control_rate * 100 if control_rate > 0 else 0
        
        better = '🟢 TEST' if test_rate > control_rate else '🔴 CONTROL'
        
        funnel_comparison.append({
            'step': step_name,
            'control_rate': control_rate,
            'test_rate': test_rate,
            'lift': lift,
            'better_performer': better
        })
        
        print(f"{step_name:20} | Control: {control_rate*100:6.2f}% | Test: {test_rate*100:6.2f}% | Lift: {lift:+6.1f}% | {better}")
    
    return funnel_rates, pd.DataFrame(funnel_comparison)

# Perform funnel analysis
funnel_rates, funnel_comparison = funnel_analysis(primary_df)


           🔄 CAMPAIGN FUNNEL ANALYSIS

📊 Funnel Conversion Rates:

--------------------------------------------------------------------------------

🎯 CONTROL CAMPAIGN:
   Impressions → Clicks:      4.86%  (3,177,233 → 154,303)
   Clicks → Searches:        41.75%  (154,303 → 64,418)
   Searches → Views:         87.51%  (64,418 → 56,370)
   Views → Carts:            66.88%  (56,370 → 37,700)
   Carts → Purchases:        40.21%  (37,700 → 15,161)
   --------------------------------------------------
   Overall Conversion:      0.4772%  (3,177,233 → 15,161)

🎯 TEST CAMPAIGN:
   Impressions → Clicks:      8.25%  (2,123,249 → 175,107)
   Clicks → Searches:        40.24%  (175,107 → 70,463)
   Searches → Views:         77.89%  (70,463 → 54,882)
   Views → Carts:            46.45%  (54,882 → 25,490)
   Carts → Purchases:        58.33%  (25,490 → 14,869)
   --------------------------------------------------
   Overall Conversion:      0.7003%  (2,123,249 → 14,869)

🚀 FUNNEL STEP PERFORMANCE C

## 6. Advanced Visualizations

In [7]:
def create_comprehensive_visualizations(df, funnel_rates, test_results):
    """
    Create comprehensive visualizations for A/B test analysis
    """
    
    # 1. Funnel Chart
    fig_funnel = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Control Campaign Funnel', 'Test Campaign Funnel'),
        specs=[[{"type": "funnel"}, {"type": "funnel"}]]
    )
    
    # Control funnel
    control_data = funnel_rates[funnel_rates['campaign'] == 'control'].iloc[0]
    fig_funnel.add_trace(go.Funnel(
        y=["Impressions", "Clicks", "Searches", "Views", "Carts", "Purchases"],
        x=[control_data['impressions'], control_data['clicks'], control_data['searches'], 
           control_data['views'], control_data['carts'], control_data['purchases']],
        name="Control",
        marker_color="lightblue"
    ), row=1, col=1)
    
    # Test funnel
    test_data = funnel_rates[funnel_rates['campaign'] == 'test'].iloc[0]
    fig_funnel.add_trace(go.Funnel(
        y=["Impressions", "Clicks", "Searches", "Views", "Carts", "Purchases"],
        x=[test_data['impressions'], test_data['clicks'], test_data['searches'], 
           test_data['views'], test_data['carts'], test_data['purchases']],
        name="Test",
        marker_color="lightcoral"
    ), row=1, col=2)
    
    fig_funnel.update_layout(title_text="Campaign Funnel Comparison", height=500)
    fig_funnel.show()
    
    # 2. Key Metrics Comparison
    fig_metrics = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Conversion Rate', 'Cost Per Acquisition', 'Click-Through Rate', 'Daily Performance'),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )
    
    # Conversion Rate
    fig_metrics.add_trace(go.Bar(
        x=['Control', 'Test'],
        y=[test_results['conversion_rate']['control_rate']*100, 
           test_results['conversion_rate']['test_rate']*100],
        name='Conversion Rate (%)',
        marker_color=['lightblue', 'lightcoral']
    ), row=1, col=1)
    
    # CPA (if available)
    if 'cpa' in test_results:
        fig_metrics.add_trace(go.Bar(
            x=['Control', 'Test'],
            y=[test_results['cpa']['control_cpa'], test_results['cpa']['test_cpa']],
            name='Cost Per Acquisition ($)',
            marker_color=['lightblue', 'lightcoral']
        ), row=1, col=2)
    
    # CTR
    fig_metrics.add_trace(go.Bar(
        x=['Control', 'Test'],
        y=[test_results['ctr']['control_ctr']*100, test_results['ctr']['test_ctr']*100],
        name='Click-Through Rate (%)',
        marker_color=['lightblue', 'lightcoral']
    ), row=2, col=1)
    
    # Daily conversion rates over time
    daily_conv = df.copy()
    daily_conv['conversion_rate'] = daily_conv['purchase'] / daily_conv['click'] * 100
    daily_conv = daily_conv.replace([np.inf, -np.inf], np.nan).dropna()
    
    for campaign in ['control', 'test']:
        campaign_data = daily_conv[daily_conv['campaign_name'] == campaign]
        fig_metrics.add_trace(go.Scatter(
            x=campaign_data['date'],
            y=campaign_data['conversion_rate'],
            mode='lines+markers',
            name=f'{campaign.title()} Daily CR',
            line=dict(color='lightblue' if campaign == 'control' else 'lightcoral')
        ), row=2, col=2)
    
    fig_metrics.update_layout(title_text="Key Performance Metrics Comparison", height=800, showlegend=True)
    fig_metrics.show()
    
    # 3. Statistical Significance Summary
    fig_significance = go.Figure()
    
    metrics = ['Conversion Rate', 'Click-Through Rate']
    if 'cpa' in test_results:
        metrics.append('Cost Per Acquisition')
    
    p_values = [test_results['conversion_rate']['p_value'], 
               test_results['ctr']['p_value']]
    if 'cpa' in test_results:
        p_values.append(test_results['cpa']['p_value'])
    
    colors = ['green' if p < 0.05 else 'red' for p in p_values]
    
    fig_significance.add_trace(go.Bar(
        x=metrics,
        y=[-np.log10(p) for p in p_values],
        marker_color=colors,
        name='Statistical Significance'
    ))
    
    fig_significance.add_hline(y=-np.log10(0.05), line_dash="dash", line_color="black", 
                              annotation_text="α = 0.05")
    
    fig_significance.update_layout(
        title="Statistical Significance Test Results (-log10 p-value)",
        yaxis_title="-log10(p-value)",
        xaxis_title="Metrics",
        height=400
    )
    fig_significance.show()

# Create visualizations
create_comprehensive_visualizations(primary_df, funnel_rates, test_results)

## 7. Sensitivity Analysis Across Scenarios

In [8]:
def sensitivity_analysis(scenarios):
    """
    Test the robustness of conclusions across different data treatment scenarios
    """
    print("\n" + "="*60)
    print("           🔬 SENSITIVITY ANALYSIS")
    print("="*60)
    print("Testing robustness of conclusions across different missing data treatments\n")
    
    sensitivity_results = {}
    
    for scenario_name, df in scenarios.items():
        print(f"\n📊 Scenario: {scenario_name.replace('_', ' ').title()}")
        print("-" * 50)
        
        # Quick conversion rate test for each scenario
        control_data = df[df['campaign_name'] == 'control']
        test_data = df[df['campaign_name'] == 'test']
        
        control_conversions = control_data['purchase'].sum()
        control_clicks = control_data['click'].sum()
        test_conversions = test_data['purchase'].sum()
        test_clicks = test_data['click'].sum()
        
        if control_clicks > 0 and test_clicks > 0:
            control_rate = control_conversions / control_clicks
            test_rate = test_conversions / test_clicks
            lift = (test_rate - control_rate) / control_rate * 100
            
            # Statistical test
            count = np.array([control_conversions, test_conversions])
            nobs = np.array([control_clicks, test_clicks])
            z_stat, p_value = proportions_ztest(count, nobs)
            
            sensitivity_results[scenario_name] = {
                'control_rate': control_rate,
                'test_rate': test_rate,
                'lift': lift,
                'p_value': p_value,
                'significant': p_value < 0.05
            }
            
            print(f"Control Conversion Rate: {control_rate*100:.2f}%")
            print(f"Test Conversion Rate: {test_rate*100:.2f}%")
            print(f"Lift: {lift:+.1f}%")
            print(f"P-value: {p_value:.6f}")
            print(f"Significant: {'✅ YES' if p_value < 0.05 else '❌ NO'}")
    
    # Summary of sensitivity analysis
    print("\n" + "="*60)
    print("           📋 SENSITIVITY SUMMARY")
    print("="*60)
    
    significant_count = sum(1 for result in sensitivity_results.values() if result['significant'])
    total_scenarios = len(sensitivity_results)
    
    print(f"\n🎯 Scenarios with significant results: {significant_count}/{total_scenarios}")
    
    if significant_count == total_scenarios:
        print("✅ ROBUST: All scenarios show significant results - conclusions are reliable")
    elif significant_count > total_scenarios/2:
        print("⚠️  MODERATE: Most scenarios show significant results - conclusions are moderately reliable")
    else:
        print("❌ SENSITIVE: Results are sensitive to missing data treatment - exercise caution")
    
    # Show range of lift estimates
    lifts = [result['lift'] for result in sensitivity_results.values()]
    print(f"\n📈 Lift Range: {min(lifts):+.1f}% to {max(lifts):+.1f}%")
    print(f"📊 Average Lift: {np.mean(lifts):+.1f}% (±{np.std(lifts):.1f}%)")
    
    return sensitivity_results

# Perform sensitivity analysis
sensitivity_results = sensitivity_analysis(analysis_scenarios)


           🔬 SENSITIVITY ANALYSIS
Testing robustness of conclusions across different missing data treatments


📊 Scenario: Complete Cases
--------------------------------------------------
Control Conversion Rate: 9.83%
Test Conversion Rate: 8.49%
Lift: -13.6%
P-value: 0.000000
Significant: ✅ YES

📊 Scenario: Median Fill
--------------------------------------------------
Control Conversion Rate: 9.82%
Test Conversion Rate: 8.64%
Lift: -12.0%
P-value: 0.000000
Significant: ✅ YES

📊 Scenario: Mean Fill
--------------------------------------------------
Control Conversion Rate: 9.83%
Test Conversion Rate: 8.64%
Lift: -12.1%
P-value: 0.000000
Significant: ✅ YES

           📋 SENSITIVITY SUMMARY

🎯 Scenarios with significant results: 3/3
✅ ROBUST: All scenarios show significant results - conclusions are reliable

📈 Lift Range: -13.6% to -12.0%
📊 Average Lift: -12.5% (±0.7%)


## 8. Business Impact & Recommendations

In [9]:
def generate_business_recommendations(test_results, funnel_comparison, sensitivity_results):
    """
    Generate comprehensive business recommendations based on statistical analysis
    """
    print("\n" + "="*80)
    print("                    📈 BUSINESS IMPACT & RECOMMENDATIONS")
    print("="*80)
    
    # Primary metric analysis
    conversion_result = test_results['conversion_rate']
    
    print("\n🎯 PRIMARY FINDING: PURCHASE CONVERSION RATE")
    print("-" * 60)
    
    if conversion_result['significant']:
        winner = 'Test' if conversion_result['test_rate'] > conversion_result['control_rate'] else 'Control'
        print(f"✅ STATISTICALLY SIGNIFICANT RESULT DETECTED")
        print(f"🏆 Winner: {winner} Campaign")
        print(f"📊 Conversion Rate Improvement: {abs(conversion_result['lift_percent']):.1f}%")
        print(f"📈 From {conversion_result['control_rate']*100:.2f}% to {conversion_result['test_rate']*100:.2f}%")
        
        # Business impact estimation
        if winner == 'Test':
            print(f"\n💰 BUSINESS IMPACT PROJECTION:")
            print(f"   • For every 1000 clicks, expect {(conversion_result['test_rate'] - conversion_result['control_rate'])*1000:.0f} additional purchases")
            print(f"   • Relative improvement of {conversion_result['lift_percent']:+.1f}% in conversion performance")
            
            recommendation = "RECOMMEND: Implement Test Campaign"
            action = "🚀 SCALE UP the Test campaign approach across all marketing efforts"
        else:
            recommendation = "RECOMMEND: Continue with Control Campaign"
            action = "⚠️  INVESTIGATE why Test campaign underperformed before making changes"
            
    else:
        print(f"❌ NO STATISTICALLY SIGNIFICANT DIFFERENCE DETECTED")
        print(f"📊 Observed difference: {conversion_result['lift_percent']:+.1f}%")
        print(f"🔬 P-value: {conversion_result['p_value']:.6f} (not significant at α=0.05)")
        
        recommendation = "RECOMMEND: No Change Needed"
        action = "🔄 Continue with current approach or run longer test for more power"
    
    print(f"\n🎯 {recommendation}")
    print(f"📋 {action}")
    
    # Funnel insights
    print("\n\n🔄 FUNNEL OPTIMIZATION OPPORTUNITIES")
    print("-" * 60)
    
    # Find biggest lift and biggest drop in funnel
    funnel_lifts = funnel_comparison.set_index('step')['lift']
    best_step = funnel_lifts.idxmax()
    worst_step = funnel_lifts.idxmin()
    
    print(f"🟢 BEST PERFORMING STEP: {best_step}")
    print(f"   • Test campaign shows {funnel_lifts[best_step]:+.1f}% improvement")
    print(f"   • Recommend: Study and replicate this success in other campaigns")
    
    print(f"\n🔴 UNDERPERFORMING STEP: {worst_step}")
    print(f"   • Test campaign shows {funnel_lifts[worst_step]:+.1f}% change")
    if funnel_lifts[worst_step] < 0:
        print(f"   • Recommend: Investigate why this step underperformed in Test campaign")
    else:
        print(f"   • This step still shows improvement in Test campaign")
    
    # Sensitivity analysis interpretation
    print("\n\n🔬 ANALYSIS RELIABILITY")
    print("-" * 60)
    
    significant_scenarios = sum(1 for r in sensitivity_results.values() if r['significant'])
    total_scenarios = len(sensitivity_results)
    
    if significant_scenarios == total_scenarios:
        reliability = "HIGH"
        confidence = "Very confident in recommendations"
    elif significant_scenarios > total_scenarios/2:
        reliability = "MODERATE"
        confidence = "Moderately confident in recommendations"
    else:
        reliability = "LOW"
        confidence = "Low confidence - need more data"
    
    print(f"📊 Analysis Reliability: {reliability}")
    print(f"🎯 Confidence Level: {confidence}")
    print(f"✅ Consistent results across {significant_scenarios}/{total_scenarios} data scenarios")
    
    # Next steps
    print("\n\n📋 RECOMMENDED NEXT STEPS")
    print("-" * 60)
    
    if conversion_result['significant'] and reliability == "HIGH":
        print("1. 🚀 IMMEDIATE: Implement winning campaign strategy")
        print("2. 📈 MONITOR: Track performance metrics closely for 2-4 weeks")
        print("3. 🔍 ANALYZE: Deep dive into what made the winning campaign successful")
        print("4. 📊 OPTIMIZE: Apply learnings to improve other marketing campaigns")
    elif not conversion_result['significant']:
        print("1. 🔄 EXTEND: Run the test longer to increase statistical power")
        print("2. 📊 INCREASE: Consider larger sample sizes for future tests")
        print("3. 🎯 FOCUS: Test more dramatic differences between campaigns")
        print("4. 📈 MONITOR: Track secondary metrics that may show clearer differences")
    else:
        print("1. 🔍 INVESTIGATE: Understand data quality issues affecting reliability")
        print("2. 🧹 IMPROVE: Enhance data collection processes")
        print("3. 🔄 REPEAT: Run the test again with better data quality controls")
        print("4. 📋 VALIDATE: Confirm results with additional testing")
    
    print("\n" + "="*80)
    print("                           ✅ ANALYSIS COMPLETE")
    print("="*80)

# Generate comprehensive business recommendations
generate_business_recommendations(test_results, funnel_comparison, sensitivity_results)


                    📈 BUSINESS IMPACT & RECOMMENDATIONS

🎯 PRIMARY FINDING: PURCHASE CONVERSION RATE
------------------------------------------------------------
✅ STATISTICALLY SIGNIFICANT RESULT DETECTED
🏆 Winner: Control Campaign
📊 Conversion Rate Improvement: 13.6%
📈 From 9.83% to 8.49%

🎯 RECOMMEND: Continue with Control Campaign
📋 ⚠️  INVESTIGATE why Test campaign underperformed before making changes


🔄 FUNNEL OPTIMIZATION OPPORTUNITIES
------------------------------------------------------------
🟢 BEST PERFORMING STEP: Impression → Click
   • Test campaign shows +69.8% improvement
   • Recommend: Study and replicate this success in other campaigns

🔴 UNDERPERFORMING STEP: View → Cart
   • Test campaign shows -30.6% change
   • Recommend: Investigate why this step underperformed in Test campaign


🔬 ANALYSIS RELIABILITY
------------------------------------------------------------
📊 Analysis Reliability: HIGH
🎯 Confidence Level: Very confident in recommendations
✅ Consistent res

## 9. Statistical Summary Report

In [10]:
def generate_executive_summary(test_results, funnel_rates):
    """
    Generate executive summary table for stakeholders
    """
    print("\n" + "="*80)
    print("                        📊 EXECUTIVE SUMMARY TABLE")
    print("="*80)
    
    # Create summary dataframe
    summary_data = []
    
    # Conversion Rate
    cr = test_results['conversion_rate']
    summary_data.append({
        'Metric': 'Purchase Conversion Rate',
        'Control': f"{cr['control_rate']*100:.2f}%",
        'Test': f"{cr['test_rate']*100:.2f}%",
        'Lift': f"{cr['lift_percent']:+.1f}%",
        'P-value': f"{cr['p_value']:.4f}",
        'Significant': '✅' if cr['significant'] else '❌',
        'Winner': 'Test' if cr['test_rate'] > cr['control_rate'] else 'Control'
    })
    
    # Click-Through Rate
    ctr = test_results['ctr']
    summary_data.append({
        'Metric': 'Click-Through Rate',
        'Control': f"{ctr['control_ctr']*100:.2f}%",
        'Test': f"{ctr['test_ctr']*100:.2f}%",
        'Lift': f"{ctr['lift_percent']:+.1f}%",
        'P-value': f"{ctr['p_value']:.4f}",
        'Significant': '✅' if ctr['significant'] else '❌',
        'Winner': 'Test' if ctr['test_ctr'] > ctr['control_ctr'] else 'Control'
    })
    
    # CPA if available
    if 'cpa' in test_results:
        cpa = test_results['cpa']
        summary_data.append({
            'Metric': 'Cost Per Acquisition',
            'Control': f"${cpa['control_cpa']:.2f}",
            'Test': f"${cpa['test_cpa']:.2f}",
            'Lift': f"{cpa['lift_percent']:+.1f}%",
            'P-value': f"{cpa['p_value']:.4f}",
            'Significant': '✅' if cpa['significant'] else '❌',
            'Winner': 'Test' if cpa['test_cpa'] < cpa['control_cpa'] else 'Control'  # Lower is better for CPA
        })
    
    # Overall funnel performance
    control_overall = funnel_rates[funnel_rates['campaign'] == 'control']['overall_conversion'].iloc[0]
    test_overall = funnel_rates[funnel_rates['campaign'] == 'test']['overall_conversion'].iloc[0]
    overall_lift = (test_overall - control_overall) / control_overall * 100
    
    summary_data.append({
        'Metric': 'Overall Funnel Conversion',
        'Control': f"{control_overall*100:.3f}%",
        'Test': f"{test_overall*100:.3f}%",
        'Lift': f"{overall_lift:+.1f}%",
        'P-value': 'See Primary',
        'Significant': '✅' if cr['significant'] else '❌',
        'Winner': 'Test' if test_overall > control_overall else 'Control'
    })
    
    # Create and display summary table
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    
    # Key takeaways
    print("\n\n🔑 KEY TAKEAWAYS:")
    print("-" * 50)
    
    if cr['significant']:
        winner = 'Test' if cr['test_rate'] > cr['control_rate'] else 'Control'
        print(f"• {winner} campaign shows statistically significant improvement")
        print(f"• Primary conversion rate improved by {abs(cr['lift_percent']):.1f}%")
        
        if winner == 'Test':
            print(f"• Recommend implementing Test campaign strategy")
        else:
            print(f"• Recommend staying with Control campaign")
    else:
        print(f"• No statistically significant difference between campaigns")
        print(f"• Consider longer testing period or larger sample size")
        print(f"• Current approach appears adequate")
    
    return summary_df

# Generate executive summary
executive_summary = generate_executive_summary(test_results, funnel_rates)


                        📊 EXECUTIVE SUMMARY TABLE
                   Metric Control   Test   Lift     P-value Significant  Winner
 Purchase Conversion Rate   9.83%  8.49% -13.6%      0.0000           ✅ Control
       Click-Through Rate   4.86%  8.25% +69.8%      0.0000           ✅    Test
     Cost Per Acquisition   $4.41  $5.02 +13.8%      0.1968           ❌ Control
Overall Funnel Conversion  0.477% 0.700% +46.8% See Primary           ✅    Test


🔑 KEY TAKEAWAYS:
--------------------------------------------------
• Control campaign shows statistically significant improvement
• Primary conversion rate improved by 13.6%
• Recommend staying with Control campaign


## 10. Conclusion

### 📋 Analysis Methodology Summary:
1. **Data Quality Assessment**: Comprehensive evaluation of data completeness and integrity
2. **Sensitivity Analysis**: Testing robustness across multiple missing data treatment scenarios
3. **Statistical Testing**: Rigorous hypothesis testing using appropriate statistical methods
4. **Funnel Analysis**: Step-by-step conversion analysis to identify optimization opportunities
5. **Business Impact**: Translation of statistical results into actionable business recommendations

### 🎯 Statistical Rigor:
- Two-proportion z-tests for conversion rate comparisons
- Confidence intervals for effect size estimation
- Multiple scenario testing for result validation
- Appropriate statistical test selection based on data characteristics

### 📈 Business Value:
This analysis provides a complete framework for data-driven campaign optimization decisions, ensuring statistical rigor while maintaining focus on practical business outcomes.